# Multipool Google Sentiment Analysis

this function will get data from google_result table
and calculate the sentiment analysis. Result is store in google_result_sentiment

crontab command
* * * * * /home/haviz/multipool/ai-case-study/run-ai-sa-google.sh >> /home/haviz/multipool/ai-sa-google.log



In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import pymysql.cursors
import sqlalchemy as sa
import nltk
import string
import os

from nltk.corpus import stopwords 
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.sql import text
from sqlalchemy.orm import sessionmaker
import pandas as pd

# nltk.download('stopwords')


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Use a pipeline as a high-level helper
from transformers import pipeline
pipe = pipeline("text-classification", model="ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa")

In [2]:
from sqlalchemy.util import deprecations
deprecations.SILENCE_UBER_WARNING = True

def execute_query(query, params=None):
    # Set your PostgreSQL connection parameters
    db_params = {
        'host': '103.82.242.92',
        'port': '5710',
        'database': 'medols',
        'user': 'postgres',
        'password': 'FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    }

    # Create a SQLAlchemy engine
    engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

    # Create a metadata object
    metadata = MetaData(bind=engine)

    # Example: Define your table
    your_table = Table('your_table', metadata,
                       Column('id', Integer, primary_key=True),
                       Column('column1', String),
                       Column('column2', String)
                       )

    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Execute the query with optional parameters
        result = session.execute(text(query), params)

        # Check if the query is a SELECT query
        is_select_query = result.returns_rows

        if is_select_query:
            # Fetch the data and return as a Pandas DataFrame
            columns = result.keys()
            fetched_data = result.fetchall()
            df = pd.DataFrame(fetched_data, columns=columns)
            # print("Fetched Data as DataFrame:")
            # print(df)
            return df
        else:
            # Get the number of rows affected for non-SELECT queries
            rows_affected = result.rowcount

            # Commit the changes to the database for non-SELECT queries
            session.commit()

            print(f"Query executed successfully. {rows_affected} rows affected.")
            return rows_affected
    except Exception as e:
        # Rollback changes if there's an error
        session.rollback()
        print(f"Error executing query: {e}")
    finally:
        # Close the session
        session.close()

In [3]:
import pandas as pd
import re
from sqlalchemy import create_engine

def stemming(comment):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in comment:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    return d_clean
    
# function case folding
def casefolding(comment):
    comment = comment.lower()
    comment = comment.strip(" ")
    comment = re.sub(r'[?|$|.|!_:")(-+,]','',comment)
    return comment

def clean_up_tag(comment):
    x_ret = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",comment).split())
    return x_ret

# Text Preprocessing, 
def text_preproc(strIn):
    # case folding
    strOut = strIn.lower()
    
    # remove numbers
    strOut = re.sub(r"\d+", "", strOut)
    
    # remote punctuation
    strOut = strOut.translate(str.maketrans("","",string.punctuation))
    
    # remove whitspace
    strOut = strOut.strip()
    
    # 
    strOut = re.sub('\s+',' ',strOut)
    return strOut

In [4]:
def process_by_df(source_ds):
    from tqdm import tqdm
    pd.options.mode.chained_assignment = None 

    # we calculate sentiment for 'title' and 'description' column

    # remove first
    source_ds['title'] = source_ds['title'].apply(clean_up_tag)
    source_ds['description'] = source_ds['description'].apply(clean_up_tag)


    # skip stemming
    source_ds['stemmed'] = source_ds['title']
    source_ds['stemmed'] = source_ds['stemmed'].apply(casefolding)
    source_ds['stemmed'] = source_ds['stemmed'].apply(text_preproc)

    sw = stopwords.words('indonesian')
    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)
    sw = stopwords.words('indonesian')

    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay

    # Create a new list and insert each element from the original list
    from tqdm import tqdm

    list_text = source_ds['stemmed'].tolist()
    new_list = []
    for i in tqdm(range( len(list_text) )):
        res = pipe(list_text[i])
        #print(res)
        new_list.append(res)

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay
    # create a list of our conditions
    for i in range(0, len(new_list)):
        # print(new_list[i][0]['label'])
        if new_list[i][0]['label'] == 'Positive':
            new_list[i][0]['Value'] = 1
        elif new_list[i][0]['label'] == 'Negative':
            new_list[i][0]['Value'] = -1
        elif new_list[i][0]['label'] == 'Neutral':
            new_list[i][0]['Value'] = 0

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']
        source_ds.at[index,'Score'] = new_list[index][0]['score']
        #source_ds.at[index,'Score'] = new_list[index][0]['Value']
        
    return source_ds


In [5]:
# query 400 rows of data query data from table
select_query = "select * \
from google_result gr \
where google_result_id not in (select google_result_id from google_result_sentiment) \
order by google_result_id desc limit 500"

source_ds = execute_query(select_query)

if len(source_ds) == 0:
    # get out, nothing to do
    print('Zero jobs, quitting now')
    quit()
    
# source_ds.head()

In [6]:
res_ds = process_by_df(source_ds)

100%|██████████| 253/253 [00:09<00:00, 25.67it/s]


In [7]:
res_ds.head()

,id,monitoring_id,source,title,description,date,status,created_date,updated_date,link,...,content_html,flag_calc,google_result_id,publish_date,is_content,is_content_error,stemmed,tokenized_tweet,Prediction,Score
0,775e5b7b-982f-470c-afaf-00bbfb22cc40,2b768b57-280b-4031-8888-6235900ce8d2,saibumi.id,As SDM Polri Jangan Percaya Imingan Bayar Seju...,As SDM Polri Jangan Percaya Lolos Rekrutmen Ha...,2023-07-11 10:30:34.000,0,2023-07-11 22:30:34.520000+00:00,2023-07-11 22:30:34.520000+00:00,https://saibumi.id/as-sdm-polri-jangan-percaya...,...,None,None,253,2023-07-11 10:30:34,None,None,as sdm polri jangan percaya imingan bayar seju...,"{imingan, sdm, uang, polri, bayar, as, percaya}",Neutral,0.997784
1,f5093c4a-c18c-45f6-99b9-7a07f7cfabb5,2b768b57-280b-4031-8888-6235900ce8d2,javanewsonline.com,As SDM Polri Jangan Percaya Lolos Javanewsonli...,As SDM Polri Jangan Percaya Lolos Rekrutmen Ha...,2023-07-11 15:30:34.000,0,2023-07-11 22:30:34.520000+00:00,2023-07-11 22:30:34.520000+00:00,https://javanewsonline.com/index.php/nasional/...,...,None,None,252,2023-07-11 15:30:34,None,None,as sdm polri jangan percaya lolos javanewsonli...,"{sdm, javanewsonline, polri, as, com, lolos, p...",Negative,0.563102
2,7d97c23a-ecd8-4fa0-8724-e7cec9a21303,2b768b57-280b-4031-8888-6235900ce8d2,mediakompeten.co.id,As SDM Polri Lolos Rekrutmen Polri Harus Bayar...,SEMARANG MEDIA KOMPETEN NEWS Asisten Kapolri b...,2023-07-11 12:30:34.000,0,2023-07-11 22:30:34.520000+00:00,2023-07-11 22:30:34.520000+00:00,https://mediakompeten.co.id/artikel/as-sdm-pol...,...,None,None,251,2023-07-11 12:30:34,None,None,as sdm polri lolos rekrutmen polri harus bayar...,"{sdm, rekrutmen, bayar, as, lolos, polri}",Neutral,0.992571
3,c59bcaae-2ba0-461d-8975-0bc4dfbddb29,2b768b57-280b-4031-8888-6235900ce8d2,heloindonesia.com,Jangan Percaya Rekrutmen Polri Bayar Pasti Dib...,Asisten Kapolri bidang Sumber Daya Manusia As ...,2023-07-11 13:30:34.000,0,2023-07-11 22:30:34.520000+00:00,2023-07-11 22:30:34.520000+00:00,https://heloindonesia.com/peristiwa/5519/janga...,...,None,None,250,2023-07-11 13:30:34,None,None,jangan percaya rekrutmen polri bayar pasti dib...,"{dibohongi, rekrutmen, bayar, percaya, polri}",Negative,0.999538
4,318aba61-96ca-4d9b-aba8-6b396ddc2d25,2b768b57-280b-4031-8888-6235900ce8d2,bimata.id,As SDM Polri Tegaskan Jangan Percaya Lolos Rek...,As SDM Polri Tegaskan Jangan Percaya Lolos Rek...,2023-07-11 12:30:34.000,0,2023-07-11 22:30:34.520000+00:00,2023-07-11 22:30:34.520000+00:00,https://bimata.id/2023/07/as-sdm-polri-tegaska...,...,None,None,249,2023-07-11 12:30:34,None,None,as sdm polri tegaskan jangan percaya lolos rek...,"{tegaskan, sdm, rekrutmen, polri, as, lolos, p...",Neutral,0.999471


In [8]:
# record result
sentiment_class = 0

# INSERT INTO public.google_result_sentiment (id, sentiment_category, score, google_result_id) VALUES(nextval('google_result_sentiment_id_seq'::regclass), 0, 0, 0);

for index, row in res_ds.iterrows():
    if row['Prediction'] == 'Positive':
        sentiment_class = 1
    elif row['Prediction'] == 'Negative':
        sentiment_class = -1
    else:
        sentiment_class = 0
        
    # sql = "INSERT INTO cekmedsos_database.ret_sentiment_result (job_id, tweet_id, sentiment_class) VALUES(%s, %s , %s);" % (str(const_job_id), row['id_str'], str(sentiment_class))
    sql = "INSERT INTO public.google_result_sentiment (id, sentiment_category, score, google_result_id) VALUES(nextval('google_result_sentiment_id_seq'::regclass), %s, %s, %s);" % (sentiment_class,row['Score'],row['google_result_id']) 
    #print(sql)
    execute_query(sql)

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 row